# Import required Libraries

* Numpy for calculations
* OpenCV for image processing, image reading and image writting
* Matplotlib for Representations

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

Importing images for Stereo Vision and initialize disparity limit and block length

In [ ]:
disparities = 64
block = 15

left = cv.imread('../images/image_4_l.png', cv.IMREAD_GRAYSCALE)
right = cv.imread('../images/image_4_r.png', cv.IMREAD_GRAYSCALE)

disparity = np.zeros(shape=(left.shape[0]-(block-1), left.shape[1]-(block-1)))

# BLock Matching Algorithms

Local methods compute disparity by considering a small neighborhood (window) around each pixel in one image and searching for the corresponding block in the other image. These methods are computationally efficient and typically used for real-time applications.

- SAD Algorithm
- SSD Algorithm
- Normalized Cross Correlation
- Census Transform

# Global Methods

Global methods solve the stereo matching problem by optimizing a global energy function. These methods aim to produce a smooth disparity map that is globally consistent across the entire image. However, they tend to be more computationally expensive compared to local methods.

- Graph Cuts
- Dynamic Programming Methods
- Semi Global Methods

# Hybrid Methods

Hybrid methods combine elements of both local and global approaches, attempting to leverage the advantages of both. 

# Neural Networks

With the rise of deep learning, there has been increasing interest in using convolutional neural networks (CNNs) and other machine learning techniques to improve stereo vision algorithms. These methods learn to predict disparities from large datasets of stereo images.


# SAD Algorithm for disparity Maps

The Sum of Absolute Differences (SAD) algorithm is one of the most widely used techniques for block-based stereo matching in computer vision, particularly for depth map creation. It compares blocks of pixels (also known as windows) between the left and right images to compute disparity, which is the pixel shift between the two images caused by the depth of objects in the scene.

In [ ]:
for i in range(left.shape[0] - block):
    print(i, "Running ...")
    for j in range(left.shape[1] - block):
        ssd = []
 
        l = left[(i) : (i + block), (j) : (j + block)]
        
        for d in range(0, disparities):
            if d + j + block >= right.shape[1]:
                break
            r = right[(i) : (i + block), (j + d) : (j + d + block)]
            try:
                ssd.append(np.sum((l[:,:]-r[:,:])))
            except:
                print("r : " , r.shape)
                print("l : ", l.shape)
        disparity[i, j] = np.argmin(ssd)

# SSD Algorithm for disparity maps

The Sum of Squared Differences (SSD) algorithm is used in stereo vision for computing the disparity between two images (usually from a stereo camera pair). Like other block-matching algorithms, SSD compares small blocks of pixels from one image (typically the left image) to corresponding blocks in the other image (typically the right image). It measures the difference between blocks using the squared differences of pixel intensities and is often used for estimating depth in stereo vision systems.

In [ ]:
for i in range(left.shape[0] - block):
    print(i, "Running ...")
    for j in range(left.shape[1] - block):
        ssd = []
 
        l = left[(i) : (i + block), (j) : (j + block)]
        
        for d in range(0, disparities):
            if d + j + block >= right.shape[1]:
                break
            r = right[(i) : (i + block), (j + d) : (j + d + block)]
            try:
                ssd.append(np.sum((l[:,:]-r[:,:])) ** 2)
            except:
                print("r : " , r.shape)
                print("l : ", l.shape)
        disparity[i, j] = np.argmin(ssd)

# Normalized Cross-Correlation (NCC)

This is a block-matching algorithm often used in stereo vision and template matching. It evaluates how similar two blocks (or windows) of pixels are by normalizing the pixel values, making it more robust to changes in lighting and contrast. Unlike SAD or SSD, which compare pixel values directly, NCC compares the relative patterns of intensity between corresponding blocks, leading to better performance in cases where images may have different brightness or contrast.

In [ ]:
disparity = np.zeros((left.shape[0], left.shape[1]), dtype=np.float32)

half_block = block // 2

for i in range(half_block, left.shape[0] - half_block):
    for j in range(half_block, left.shape[1] - half_block):
        
        left_block = left[i - half_block:i + half_block + 1, j - half_block:j + half_block + 1]
        mean_left = np.mean(left_block)
        
        normalized_left = left_block - mean_left
        
        best_ncc = -1
        best_disparity = 0
        
        for d in range(disparities):
            if j - d - half_block >= 0:
                right_block = right[i - half_block:i + half_block + 1, j - d - half_block:j - d + half_block + 1]
                mean_right = np.mean(right_block)
                
                normalized_right = right_block - mean_right
                
                numerator = np.sum(normalized_left * normalized_right)
                
                denominator = np.sqrt(np.sum(normalized_left ** 2) * np.sum(normalized_right ** 2))
                
                if denominator > 0:
                    ncc_value = numerator / denominator
                    
                    if ncc_value > best_ncc:
                        best_ncc = ncc_value
                        best_disparity = d
        
        disparity[i, j] = best_disparity

# Census ALgorithm

 This is a method used in stereo vision for disparity estimation. It encodes local intensity information of an image into a binary representation, providing robustness against illumination changes, noise, and other variations. This makes it a popular choice for disparity estimation in various computer vision tasks, including depth mapping.

In [ ]:
def census_transform(image, window_size):
    height, width = image.shape
    half_window = window_size // 2
    census_image = np.zeros((height, width), dtype=np.uint32)

    for i in range(half_window, height - half_window):
        for j in range(half_window, width - half_window):
            center_pixel = image[i, j]
            binary_pattern = 0
            
            for m in range(-half_window, half_window + 1):
                for n in range(-half_window, half_window + 1):
                    if not (m == 0 and n == 0):
                        neighbor_pixel = image[i + m, j + n]
                        binary_pattern <<= 1
                        if neighbor_pixel < center_pixel:
                            binary_pattern |= 1
            
            census_image[i, j] = binary_pattern

    return census_image

left_census = census_transform(left, block)
right_census = census_transform(right, )

height, width = left.shape
disparity = np.zeros((height, width), dtype=np.float32)

for i in range(height):
    for j in range(width):
        min_hamming_distance = float('inf')
        best_disparity = 0

        for d in range(disparities):
            if j - d >= 0: 
                hamming_dist = np.bitwise_xor(left_census[i, j], right_census[i, j - d]).sum()
                if hamming_dist < min_hamming_distance:
                    min_hamming_distance = hamming_dist
                    best_disparity = d

        disparity[i, j] = best_disparity